# Maze Experiment Introduction

In this notebook, we will be running the MDP_Model on the Maze problem. This 2D simulation builds on a `gym-maze` package that can be found here: https://github.com/MattChanTK/gym-maze. Before beginning this simulation, please be sure to install the relevant packages on the github **Installation** section (pygame and numpy are also required)!

If you haven't done so already, you can run the following cell. It it recommended to start a new python environment before doing so, to avoid any incompatibilities.

In [71]:
# pip install -r ../requirements.txt

### Importing general modules

In [72]:
# Append system paths
import sys
sys.path.append("../gym-maze/")
sys.path.append("../mrl/")

In [73]:
# Load libraries
import numpy as np
import pandas as pd
import math
import random
import gym
import gym_maze

First, a quick demonstration about what the gym environment we use. Essentially, there is an agent that starts on the upper left cell, then keeps taking steps in the maze until it reaches the end point. Here is a simulation to demonstrate:

In [ ]:
# Initializing the gym environment for a maze

env = gym.make("maze-sample-5x5-v0")

# Set number of iterations
n_iterations = 20

# Running the maze
observation = env.reset()
for _ in range(n_iterations):
    try:
        env.render()
        action = env.action_space.sample() # your agent here (this takes random actions)
        observation, reward, done, info = env.step(action)
        if done:
          observation = env.reset()
    except ValueError:
        pass
env.close()

### The Maze environment

Here, we will observe paths of the agent walking through the Maze. Each time the agent is in a cell, we observe only a random 2D point in the cell and a rewards (here denoted RISK, $=-1/A$, where $A$ is the number of cells, except in the goal cell $= 1$). At each step, the agent can take action up, down, left, right and will transition to a random 2D point in the next cell, unless there is wall. Then, it stays in the same cell, within a new random 2D points. In particular, we do not know the cells, walls, or dynamics of the Maze. We only observe paths. The goal is to learn the structure of the Maze, which is the minimal representation of this continuous state space MDP.

In [75]:
#actions are encoded as follows:
actions = {0: 'Up', 1: 'Down', 2: 'Right', 3: 'Left'}

#### Importing Modules

Loading relevant packages and functions - make sure to change the `sys.path.append` line with the relevant directory that contains the MDP Algorithm.

In [76]:
import matplotlib.pyplot as plt

#Importing MRL specific libraries
from mdp_utils import *
from clustering import *
from model import MDP_model
from maze_functions import createSamples, opt_maze_trajectory, opt_model_trajectory, policy_accuracy, \
    get_maze_transition_reward, plot_paths
from testing import cluster_size, next_clusters, training_value_error, purity, plot_features, testing_value_error

mazes = {1: 'maze-v0',
         2: 'maze-sample-3x3-v0',
         3: 'maze-random-3x3-v0',
         4: 'maze-sample-5x5-v0',
         5: 'maze-random-5x5-v0',
         6: 'maze-sample-10x10-v0',
         7: 'maze-random-10x10-v0',
         8: 'maze-sample-100x100-v0',
         9: 'maze-random-100x100-v0',
         10: 'maze-random-10x10-plus-v0', # has portals 
         11: 'maze-random-20x20-plus-v0', # has portals 
         12: 'maze-random-30x30-plus-v0'} # has portals 

#### Creating Samples

Now selecting the environment parameters: here, we decide how many paths in the maze we generate (`N`), the number of steps (horizon) of each path (`T_max`), and the maze that we want the agent to run through.

`reseed`' is set to `True` iif we change the randomness seed every path.

`r` dictates the action policy of the agent. At each step, the agent takes a step in the optimal direction towards the goal with probability $1-r$ and a random direction with probability $r$.

We select here a $5\times 5$ Maze, generate $200$ paths, each of horizon $50$, with a policy that takes half of the times the optimal direction and a random direction the other half. You can verify here the rewards are $-0.04 = -\frac{1}{5\times 5}$ for non-goal states.

In [77]:
# Setting Parameters
N = 200
T_max = 50
r = 0.5
maze = mazes[4]

In [ ]:
df = createSamples(N, T_max, maze, r, reseed=True)
df.head()

The resulting `FEATURE_0` and `FEATURE_1` are the `x` and `y` coordinates respectively, while `ACTION` corresponds to the {0: 'Up', 1: 'Down', 2: 'Right', 3: 'Left'} directions. `RISK` is the reward (`1` if the endstate goal is reached, otherwise $-1/A$, with $A$ being the size of the maze).

That's how the transition data looks like (dashed lines are not visible to the algorithm):

In [ ]:
plot_paths(df,2)

Out of the `N` generated paths, here's how many reached the goal state:

In [ ]:
df.loc[df['ACTION']=='None']['ID'].count()

#### Fitting to Algorithm (Cross Validation (CV) Example)

We now run the algorithm on the generate data set.
This is an example of training with cross validation! For faster training, simply change `m.fit_CV` to `m.fit`. 

Here is a breakdown of MRL parameters:
- `max_k`: a cap of the number of clusters (states) we allow. *Further Notes:* This will then be am upper bound on the number of splits MRL does (and therefore on runtime). In this environment, the initial clustering, base on rewards will lead two clusters (end state and all others). The expected optimal `max_k` should be then be larger than the total number of cells of the Maze to be able to recover the minimal representation.
- `pfeatures`: how many features we have in the dataset, in this case 2 (coordinates x and y). 
- `classification`: represents the type of classifier we want to use when splitting clusters. Options for this classifier include `'DecisionTreeClassifier'`, `'LogisticRegression'`, `'RandomForestClassifier'`, `'MLPClassifier'`, and `'AdaBoostClassifier'`. *Further Notes:* Think of this as the hypothesis class. In our experiment decision trees work better.
- `split_classifier_params`: passes in the arguments necessary to the split classifier. *Further Notes:* For example for decision trees, this includes random_state and max_depth. 
- `clustering`: indicates the method used to form the initial clusters (based on rewards), with options of `'Agglomerative'`, `'KMeans'`, or `'Birch'`. 
- `n_clusters`: can be passed in if we want to fix the number of initial clusters (based on rewards). This is useful in the case of non-discrete rewards. In the case of discrete rewards, as for the Maze env, we typically use `'Agglomerative'` cluster. In this case a `distance_threshold` must also be passed in to determine the minimal distance between the rewards of two clusters (read `sklearn.cluster` documentation on `AgglomerativeClustering` for more details). 
- `precision_thresh`: determines the minimum decrease in value error necessary for the model to determine that a split gives a better clustering/representation.  *Further Notes:* This value attempts to limit model complexity when improvements becomes only incremental.
- `eta`: sets a maximum incoherence threshold for a valid representation. *Further Notes:* Incoherence is defined as the the number of points in each cluster-action pair that do not go to the majority next cluster (transition) observed in the data; see definition in the paper. During training, any clustering/representation that results in a maximum incoherence above `eta*sqrt(n)/c`, where `n` is the number of data points given and `c` is the number of clusters at this current split, will be disregarded as too incoherent when selecting the number of clusters for the learnt representation.
- `th`: is a threshold on the number of incoherences in a cluster/state to trigger a split.
- `gamma`: is the MDP discount factor used when calculating values (in particular evaluating value error).
- `h` is the number of time steps (horizon) on which to evaluate the value error. When `h = -1`, we evaluate over an infinite horizon, the whole path.
- `cv` is the number of folds for cross validation, only relevant when you run `model.fit_CV()`. 


In [ ]:
# Setting parameters for model fitting
max_k = 20
pfeatures = 2
classification = 'DecisionTreeClassifier'
split_classifier_params = {'random_state':0, 'max_depth':2}
clustering = 'Agglomerative'
n_clusters = None
distance_threshold = 0.5 #for Agglomerative clustering
random_state = 0
h = -1
gamma = 1
cv = 5
th = 0
eta = 25
precision_thresh = 1e-14

m = MDP_model()
m.fit_CV(df, # df: dataframe in the format ['ID', 'TIME', ...features..., 'RISK', 'ACTION']
    pfeatures, # int: number of features
    h, # int: time horizon (# of actions we want to optimize)
    gamma, # discount factor
    max_k, # int: number of iterations
    distance_threshold, # clustering diameter for Agglomerative clustering
    cv, # number for cross validation
    th, # splitting threshold
    eta, # incoherence threshold, calculated by eta*sqrt(datapoints)/clusters
    precision_thresh, # precision threshold
    classification, # classification method
    split_classifier_params, # classifier params
    clustering,# clustering method from Agglomerative, KMeans, and Birch
    n_clusters, # number of clusters for KMeans
    random_state,
    plot=True)

#### Visualizing the learnt MRL representation

First, let us see what MDP states/clusters MRL learnt, and compare it with the actual cells of the Maze.
We make a special class `Maze_Model_Visualizer` to visualize various aspects of a learnt MRL's maze representation.

In [ ]:
from maze_functions import Maze_Model_Visualizer
vis = Maze_Model_Visualizer(m)

vis.plot_features()
vis.plot_features(rep = 'OG_CLUSTER', title = 'Cells of the Maze')

We can also visualize a given cluster and its learnt transitions

In [ ]:
vis.plot_features() #we first create the figure to show the cluster in
#Show a given cluster
vis.show_cluster(cluster = 3, color = 'black')
#Show a given point
vis.show_point(point = (2.5,-2.2), color = 'red', s=50)


Out of the constructed states (discretization), we can now build an MDP model (see paper) and use it to predict values and learn an optimal policy.

When building the MDP model from the discretization, we incorporate some robustness checks with following paramters: `min_action_obs`, `min_action_purity`, `alpha`, `beta`. When we observe data transitions from state $s$ to $s'$ under action $a$, a transition in built in the MDP only if it passes the following tests:
- *Minimal Number of Observations:* Number of observations $s \rightarrow_a s'$ is $\geq$ `min_action_obs`.
- *Minimal Ratio of Agreement in the State:* Among the data points in state $s$, taking action $a$, at least a ratio of `min_action_purity` indeed transitioned to $s'$.
- *Hypothesis test:* We assume null hypothesis that $\leq$ `beta` ratio of our data $(s,a)$ is actually going into $s'$, and seek to reject this hypothesis with significance p<=`alpha`. If we fail to reject, then we remove this cluster-action pair.

In [84]:
#First, we build an MDP representation and compute its optimal policy
m.solve_MDP(min_action_obs=-1, alpha = 0.2, beta = 0.85, min_action_purity=0.5)

We can visualize the constructed MDP with these tools

In [ ]:
#We can now visualize a given transition
#origine cluster is in black, and the one it transition to is in red
vis.plot_features()
print('Transition for action', actions[1])
vis.show_transition(cluster=10,action=1)
#Observe the details how the constructed transition (# of points, purity, etc)
vis.transition_details(cluster=10,action=1)
#and now visualize the learnt optimal policy
vis.policy()

We now simulate a path in the Maze with the learnt policy

In [ ]:
vis.simulate_opt_policy(maze)

We can evaluate how well the model predict values of unseen data points

In [ ]:
df_test = createSamples(100, T_max, maze, 0.3, reseed=True) #new unseen trajectories
val_error = testing_value_error(df_test, m.df_trained, m.m, m.pfeatures, relative=False, h=-1); #value error on these points
print('Value error on test set:', val_error)

... and how well it learns the states of the minimal representation of the Maze 

In [ ]:
print('MRL learnt states accuracy, as compared with Maze cells:', m.clus_pred_accuracy)

## Model Performance as a Function of Data Size

We now investigate how well MRL recovers the minimal representation with growing data size $N$.

In [ ]:
# random.seed(0)
N = 200 #Max data size
T_max = 25 #Max number of steps in each trajectory
r = 0.5 #percentage of optimal action is data generation policy
maze = mazes[4]

df = createSamples(N, T_max, maze, r, reseed=True)

Now, we want to see how well the model trains on different subsets of the data, starting with when it only has access to the first 10 paths, `N=10`, all the way to all 200 paths, `N=200`.

In [ ]:
# random.seed(0)
# Setting MRL params
max_k = 25
classification = 'DecisionTreeClassifier'
split_classifier_params = {'random_state':0, 'max_depth':2}
clustering = 'Agglomerative'
n_clusters = None
distance_threshold = 0.5
random_state = 0
pfeatures = 2
gamma = 1
actions = [0, 1, 2, 3]
h = -1
cv = 5
th = 0
eta = 25
precision_thresh = 1e-14

#Data size
Ns = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, 170, 200]
df_full = df.copy()

models=[]

# Training models 
for n in Ns:
    df_small = df_full.loc[df_full['ID']<n]
    
    m = MDP_model()
    print('Training for N=', n, ' .....')
    m.fit(df_small, # df: dataframe in the format ['ID', 'TIME', ...features..., 'RISK', 'ACTION']
        pfeatures, # int: number of features
        h, # int: time horizon (# of actions we want to optimize)
        gamma, # discount factor
        max_k, # int: number of iterations
        distance_threshold, # clustering diameter for Agglomerative clustering
        cv, # number for cross validation
        th, # splitting threshold
        eta, # incoherence threshold
        precision_thresh, # precision threshold
        classification, # classification method
        split_classifier_params, # classification params
        clustering,# clustering method from Agglomerative, KMeans, and Birch
        n_clusters, # number of clusters for KMeans
        random_state,
        plot=False,
        optimize=True)
    models.append(m)

### Value Errors

We evaluate training and testing value error as a function of data size MRL was trained on. As a reminder these errors are computed by how well can MRL's model predict the value of a given path in the continuous state space.

In [ ]:
df_test.iloc[0]['FEATURE_0']

In [ ]:
from testing import testing_value_error

random.seed(0)
np.random.seed(0)
# Creating a test set with same parameters as training set
N = 200
T_max = 25
r = 0.5
maze = mazes[4]
df_test = createSamples(N, T_max, maze, r, reseed=True)

# training and testing value errors: 
training_errors = []
testing_errors = []
for m in models: 
    tr_err = m.training_error.loc[m.training_error['Clusters']==m.opt_k]['Error'].min()
    te_err = testing_value_error(df_test, m.df_trained, m.m, m.pfeatures, gamma, relative=False, h=-1)
    training_errors.append(tr_err)
    testing_errors.append(te_err)

fig1, ax1 = plt.subplots()
ax1.plot(Ns, training_errors, label='Training Error')
ax1.plot(Ns, testing_errors, label='Testing Error')
ax1.set_title('Testing and Training Errors by N')
ax1.set_xlabel('N training data size')
ax1.set_ylabel('Error')
ax1.legend()

### Accuracies

We also want to measure both training and testing state classification accuracies, measured as how well the model learns and maps each point to the correct original cell of the maze, which correspond to the minimal representation.

In [ ]:
from testing import generalization_accuracy

tr_acc, test_acc = generalization_accuracy(models, df_test, Ns)

### Optimality Gap

We measure here the value optimality gap: Given a point that start randomly in the starting state, we want to difference between the total collected value by the optimal policy and the collected value by MRL's model's prescribed policy. When simulating each policy, the point progresses through the maze according to the actual maze dynamics; the only difference lies in what sequence of actions it is given.

We will once again plot the change in this value as data size increases.

In [ ]:
from maze_functions import get_maze_MDP, get_maze_transition_reward, value_diff

random.seed(0)
# Set Parameters
P, R = get_maze_MDP(maze)
K = 100 #number of simulation of the found policy to estimate its expected value
f, rw = get_maze_transition_reward(maze)

betas = np.array(Ns)/max(Ns) *0.6+0.3
for i,model in enumerate(models):
    model.solve_MDP(min_action_obs=-1, alpha = 0.2, beta = betas[i])

opt_gap = value_diff(models, Ns, K, T_max, P, R, f, rw)
fig1, ax1 = plt.subplots()
ax1.plot(Ns, opt_gap)
ax1.set_title('Optimality Gap by Data Size N')
ax1.set_xlabel('N training data size')
ax1.set_ylabel('|V_alg-V*|')
plt.show()